In [1]:
import numpy as np
import cv2
import os
from skimage import color
import cv2
import numpy as np
from scipy.signal import convolve2d

def calcula_lpq_histograma(espectrograma, winSize=5, freqestim=1):
    """
    Calcula o histograma LPQ (Local Phase Quantization) para um espectrograma.

    Args:
        espectrograma (np.ndarray): O espectrograma de entrada com forma (altura, largura, canais).
        winSize (int, optional): O tamanho da janela para o cálculo do LPQ. Defaults to 3.
        freqestim (int, optional): O método de estimação de frequência. Defaults to 1 (STFT uniforme).

    Returns:
        np.ndarray: O histograma LPQ com 59 colunas.
    """

    rho = 0.90  # Parâmetro de ponderação para o cálculo da resposta em frequência

    STFTalpha = 1 / winSize  # Parâmetro alpha para as abordagens STFT
    sigmaS = (winSize - 1) / 4  # Desvio padrão para a janela gaussiana STFT
    sigmaA = 8 / (winSize - 1)  # Desvio padrão para os filtros de quadratura de derivada gaussiana

    convmode = 'valid'  # Modo de convolução

    espectrograma = np.float64(espectrograma)  # Converte o espectrograma para double
    r = (winSize - 1) / 2  # Raio da janela
    x = np.arange(-r, r + 1)[np.newaxis]  # Coordenadas espaciais da janela

    # Calcula o LPQ para cada canal do espectrograma
    lpq_histogramas = []
    for canal in range(espectrograma.shape[2]):
        # Extrai o canal atual
        imagem_canal = espectrograma[:, :, canal]

        # Calcula a resposta em frequência usando STFT uniforme
        if freqestim == 1:
            w0 = np.ones_like(x)
            w1 = np.exp(-2 * np.pi * x * STFTalpha * 1j)
            w2 = np.conj(w1)

            # Aplica os filtros STFT
            filterResp1 = convolve2d(convolve2d(imagem_canal, w0.T, convmode), w1, convmode)
            filterResp2 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w0, convmode)
            filterResp3 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w1, convmode)
            filterResp4 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w2, convmode)

            # Cria a matriz de resposta em frequência
            freqResp = np.dstack([filterResp1.real, filterResp1.imag,
                                  filterResp2.real, filterResp2.imag,
                                  filterResp3.real, filterResp3.imag,
                                  filterResp4.real, filterResp4.imag])

        # Quantização e cálculo dos códigos LPQ
        inds = np.arange(freqResp.shape[2])[np.newaxis, np.newaxis, :]
        LPQdesc = ((freqResp > 0) * (2 ** inds)).sum(2)

        # Calcula o histograma LPQ
        hist, bins = np.histogram(LPQdesc.flatten(), bins=256, range=(0, 256), density=True)
        lpq_histogramas.append(hist)

    # Combina os histogramas LPQ de todos os canais
    lpq_histograma = np.concatenate(lpq_histogramas)

    return lpq_histograma

# Diretórios
diretorios = ['base/fold1', 'base/fold2', 'base/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LPQ
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem)  # Ler a imagem em RGB
        hist.append(calcula_lpq_histograma(imagem))
        tags.append(j // 30 + 1)  # Tag para cada imagem

# Exemplo de uso
hist_ori = np.array(hist)
tags_ori = np.array(tags)


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'kernel': ['linear', 'rbf', 'poly']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return (mean_accuracy, std_accuracy, 
            mean_precision, std_precision, 
            mean_recall, std_recall, 
            mean_f1_score, std_f1_score)

# Realizar a validação cruzada
(mean_accuracy, std_accuracy, 
 mean_precision, std_precision, 
 mean_recall, std_recall, 
 mean_f1_score, std_f1_score) = kfoldcv(hist_ori, tags_ori, 3)

print(f'Acurácia média: {mean_accuracy:.2f}% (Desvio padrão: {std_accuracy:.2f}%)')
print(f'Precisão média: {mean_precision:.2f}% (Desvio padrão: {std_precision:.2f}%)')
print(f'Recall médio: {mean_recall:.2f}% (Desvio padrão: {std_recall:.2f}%)')
print(f'F1-score médio: {mean_f1_score:.2f}% (Desvio padrão: {std_f1_score:.2f}%)')


Acurácia média: 76.89% (Desvio padrão: 2.91%)
Precisão média: 77.99% (Desvio padrão: 3.32%)
Recall médio: 76.89% (Desvio padrão: 2.91%)
F1-score médio: 76.65% (Desvio padrão: 3.06%)


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Função para realizar a validação cruzada com KNN
def kfoldcv_knn(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search do KNN
        param_grid = {
            'n_neighbors': [3, 5, 7, 9, 11, 13],
            'metric': ['canberra', 'minkowski', 'euclidean', 'manhattan']
        }

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_knn.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return mean_accuracy, std_accuracy, mean_precision, std_precision, mean_recall, std_recall, mean_f1_score, std_f1_score

# Realizar a validação cruzada com KNN
mean_accuracy, std_accuracy, mean_precision, std_precision, mean_recall, std_recall, mean_f1_score, std_f1_score = kfoldcv_knn(hist_ori, tags_ori, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')
print(f'Precisão média: {mean_precision:.2f}%')
print(f'Desvio padrão da precisão: {std_precision:.2f}%')
print(f'Recall médio: {mean_recall:.2f}%')
print(f'Desvio padrão do recall: {std_recall:.2f}%')
print(f'F1-score médio: {mean_f1_score:.2f}%')
print(f'Desvio padrão do F1-score: {std_f1_score:.2f}%')


Acurácia média: 69.00%
Desvio padrão da acurácia: 1.78%
Precisão média: 69.37%
Desvio padrão da precisão: 1.75%
Recall médio: 69.00%
Desvio padrão do recall: 1.78%
F1-score médio: 68.66%
Desvio padrão do F1-score: 1.88%


In [6]:
from sklearn.ensemble import RandomForestClassifier

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    
    accuracies = []
    
    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Treinar o modelo SVM
        rf_model = RandomForestClassifier(n_estimators=4000, random_state=0, n_jobs=-1)
        rf_model.fit(X_train, y_train)
         
        # Fazer previsões e calcular a acurácia
        y_pred = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist_ori, tags_ori, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')

Acurácia média: 71.00
Desvio padrão da acurácia: 2.16


In [7]:
import numpy as np
import cv2
import os
from skimage import color
import cv2
import numpy as np
from scipy.signal import convolve2d

def calcula_lpq_histograma(espectrograma, winSize=5, freqestim=1):
    """
    Calcula o histograma LPQ (Local Phase Quantization) para um espectrograma.

    Args:
        espectrograma (np.ndarray): O espectrograma de entrada com forma (altura, largura, canais).
        winSize (int, optional): O tamanho da janela para o cálculo do LPQ. Defaults to 3.
        freqestim (int, optional): O método de estimação de frequência. Defaults to 1 (STFT uniforme).

    Returns:
        np.ndarray: O histograma LPQ com 59 colunas.
    """

    rho = 0.90  # Parâmetro de ponderação para o cálculo da resposta em frequência

    STFTalpha = 1 / winSize  # Parâmetro alpha para as abordagens STFT
    sigmaS = (winSize - 1) / 4  # Desvio padrão para a janela gaussiana STFT
    sigmaA = 8 / (winSize - 1)  # Desvio padrão para os filtros de quadratura de derivada gaussiana

    convmode = 'valid'  # Modo de convolução

    espectrograma = np.float64(espectrograma)  # Converte o espectrograma para double
    r = (winSize - 1) / 2  # Raio da janela
    x = np.arange(-r, r + 1)[np.newaxis]  # Coordenadas espaciais da janela

    # Calcula o LPQ para cada canal do espectrograma
    lpq_histogramas = []
    for canal in range(espectrograma.shape[2]):
        # Extrai o canal atual
        imagem_canal = espectrograma[:, :, canal]

        # Calcula a resposta em frequência usando STFT uniforme
        if freqestim == 1:
            w0 = np.ones_like(x)
            w1 = np.exp(-2 * np.pi * x * STFTalpha * 1j)
            w2 = np.conj(w1)

            # Aplica os filtros STFT
            filterResp1 = convolve2d(convolve2d(imagem_canal, w0.T, convmode), w1, convmode)
            filterResp2 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w0, convmode)
            filterResp3 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w1, convmode)
            filterResp4 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w2, convmode)

            # Cria a matriz de resposta em frequência
            freqResp = np.dstack([filterResp1.real, filterResp1.imag,
                                  filterResp2.real, filterResp2.imag,
                                  filterResp3.real, filterResp3.imag,
                                  filterResp4.real, filterResp4.imag])

        # Quantização e cálculo dos códigos LPQ
        inds = np.arange(freqResp.shape[2])[np.newaxis, np.newaxis, :]
        LPQdesc = ((freqResp > 0) * (2 ** inds)).sum(2)

        # Calcula o histograma LPQ
        hist, bins = np.histogram(LPQdesc.flatten(), bins=256, range=(0, 256), density=True)
        lpq_histogramas.append(hist)

    # Combina os histogramas LPQ de todos os canais
    lpq_histograma = np.concatenate(lpq_histogramas)

    return lpq_histograma

# Diretórios
diretorios = ['harmonico/fold1', 'harmonico/fold2', 'harmonico/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LPQ
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem)  # Ler a imagem em RGB
        hist.append(calcula_lpq_histograma(imagem))
        tags.append(j // 30 + 1)  # Tag para cada imagem

# Exemplo de uso
hist_har = np.array(hist)
tags_har = np.array(tags)


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'kernel': ['linear', 'rbf', 'poly']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return (mean_accuracy, std_accuracy, 
            mean_precision, std_precision, 
            mean_recall, std_recall, 
            mean_f1_score, std_f1_score)

# Realizar a validação cruzada
(mean_accuracy, std_accuracy, 
 mean_precision, std_precision, 
 mean_recall, std_recall, 
 mean_f1_score, std_f1_score) = kfoldcv(hist_har, tags_har, 3)

print(f'Acurácia média: {mean_accuracy:.2f}% (Desvio padrão: {std_accuracy:.2f}%)')
print(f'Precisão média: {mean_precision:.2f}% (Desvio padrão: {std_precision:.2f}%)')
print(f'Recall médio: {mean_recall:.2f}% (Desvio padrão: {std_recall:.2f}%)')
print(f'F1-score médio: {mean_f1_score:.2f}% (Desvio padrão: {std_f1_score:.2f}%)')


Acurácia média: 74.67% (Desvio padrão: 1.91%)
Precisão média: 75.99% (Desvio padrão: 1.52%)
Recall médio: 74.67% (Desvio padrão: 1.91%)
F1-score médio: 74.50% (Desvio padrão: 2.00%)


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Função para realizar a validação cruzada com KNN
def kfoldcv_knn(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search do KNN
        param_grid = {
            'n_neighbors': [3, 5, 7, 9, 11, 13],
            'metric': ['canberra', 'minkowski', 'euclidean', 'manhattan']
        }

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_knn.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return mean_accuracy, std_accuracy, mean_precision, std_precision, mean_recall, std_recall, mean_f1_score, std_f1_score

# Realizar a validação cruzada com KNN
mean_accuracy, std_accuracy, mean_precision, std_precision, mean_recall, std_recall, mean_f1_score, std_f1_score = kfoldcv_knn(hist_har, tags_har, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')
print(f'Precisão média: {mean_precision:.2f}%')
print(f'Desvio padrão da precisão: {std_precision:.2f}%')
print(f'Recall médio: {mean_recall:.2f}%')
print(f'Desvio padrão do recall: {std_recall:.2f}%')
print(f'F1-score médio: {mean_f1_score:.2f}%')
print(f'Desvio padrão do F1-score: {std_f1_score:.2f}%')


Acurácia média: 60.44%
Desvio padrão da acurácia: 1.85%
Precisão média: 61.58%
Desvio padrão da precisão: 1.58%
Recall médio: 60.44%
Desvio padrão do recall: 1.85%
F1-score médio: 60.21%
Desvio padrão do F1-score: 1.51%


In [10]:
from sklearn.ensemble import RandomForestClassifier

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    
    accuracies = []
    
    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Treinar o modelo SVM
        rf_model = RandomForestClassifier(n_estimators=4000, random_state=0, n_jobs=-1)
        rf_model.fit(X_train, y_train)
         
        # Fazer previsões e calcular a acurácia
        y_pred = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist_ori, tags_ori, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')

Acurácia média: 71.00
Desvio padrão da acurácia: 2.16


In [11]:
import numpy as np
import cv2
import os
from skimage import color
import cv2
import numpy as np
from scipy.signal import convolve2d

def calcula_lpq_histograma(espectrograma, winSize=5, freqestim=1):
    """
    Calcula o histograma LPQ (Local Phase Quantization) para um espectrograma.

    Args:
        espectrograma (np.ndarray): O espectrograma de entrada com forma (altura, largura, canais).
        winSize (int, optional): O tamanho da janela para o cálculo do LPQ. Defaults to 3.
        freqestim (int, optional): O método de estimação de frequência. Defaults to 1 (STFT uniforme).

    Returns:
        np.ndarray: O histograma LPQ com 59 colunas.
    """

    rho = 0.90  # Parâmetro de ponderação para o cálculo da resposta em frequência

    STFTalpha = 1 / winSize  # Parâmetro alpha para as abordagens STFT
    sigmaS = (winSize - 1) / 4  # Desvio padrão para a janela gaussiana STFT
    sigmaA = 8 / (winSize - 1)  # Desvio padrão para os filtros de quadratura de derivada gaussiana

    convmode = 'valid'  # Modo de convolução

    espectrograma = np.float64(espectrograma)  # Converte o espectrograma para double
    r = (winSize - 1) / 2  # Raio da janela
    x = np.arange(-r, r + 1)[np.newaxis]  # Coordenadas espaciais da janela

    # Calcula o LPQ para cada canal do espectrograma
    lpq_histogramas = []
    for canal in range(espectrograma.shape[2]):
        # Extrai o canal atual
        imagem_canal = espectrograma[:, :, canal]

        # Calcula a resposta em frequência usando STFT uniforme
        if freqestim == 1:
            w0 = np.ones_like(x)
            w1 = np.exp(-2 * np.pi * x * STFTalpha * 1j)
            w2 = np.conj(w1)

            # Aplica os filtros STFT
            filterResp1 = convolve2d(convolve2d(imagem_canal, w0.T, convmode), w1, convmode)
            filterResp2 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w0, convmode)
            filterResp3 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w1, convmode)
            filterResp4 = convolve2d(convolve2d(imagem_canal, w1.T, convmode), w2, convmode)

            # Cria a matriz de resposta em frequência
            freqResp = np.dstack([filterResp1.real, filterResp1.imag,
                                  filterResp2.real, filterResp2.imag,
                                  filterResp3.real, filterResp3.imag,
                                  filterResp4.real, filterResp4.imag])

        # Quantização e cálculo dos códigos LPQ
        inds = np.arange(freqResp.shape[2])[np.newaxis, np.newaxis, :]
        LPQdesc = ((freqResp > 0) * (2 ** inds)).sum(2)

        # Calcula o histograma LPQ
        hist, bins = np.histogram(LPQdesc.flatten(), bins=256, range=(0, 256), density=True)
        lpq_histogramas.append(hist)

    # Combina os histogramas LPQ de todos os canais
    lpq_histograma = np.concatenate(lpq_histogramas)

    return lpq_histograma

# Diretórios
diretorios = ['percussivo/fold1', 'percussivo/fold2', 'percussivo/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LPQ
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem)  # Ler a imagem em RGB
        hist.append(calcula_lpq_histograma(imagem))
        tags.append(j // 30 + 1)  # Tag para cada imagem

# Exemplo de uso
hist_har = np.array(hist)
tags_har = np.array(tags)
